In [1]:
import sys
sys.path.append('../')

import retrieval_bot
from config import corpus_path

## pair corpus

In [2]:
from retrieval_bot import PairCorpus

corpus = PairCorpus(corpus_path)
print('num pairs = {}'.format(len(corpus)))

num pairs = 5903206


## cohesion

In [3]:
from retrieval_bot.tokenizer import CohesionScore

corpus.iter_pair = False
cohesion_trainer = CohesionScore(debug=False)
#scores = cohesion_trainer.train_and_scores(corpus)
#cohesion_trainer.save('../tmp/cohesion')
cohesion_trainer.load('../tmp/cohesion')
scores = cohesion_trainer.scores()

## tokenizer

In [4]:
from retrieval_bot.tokenizer import MaxScoreTokenizer

tokenizer = MaxScoreTokenizer(scores=scores)
tokenizer.tokenize('아니지금어디냐고?왜아직도안와?')

['아니', '지금', '어디', '냐', '고?', '왜', '아직도', '안와?']

## vectorizer

In [5]:
from retrieval_bot.vectorizer import Vectorizer

corpus.limit_pairs = -1
corpus.iter_pair = False
vectorizer = Vectorizer(tokenizer=tokenizer, min_tf=2)
# vectorizer = vectorizer.fit(corpus)
# vectorizer.save('../tmp/vectorizer.vocab')
vectorizer.load('../tmp/vectorizer.vocab')
len(vectorizer.vocabulary_)

17761

## send2reply

In [6]:
from retrieval_bot.db import Send2Reply

corpus.iter_pair = True
send2reply = Send2Reply()
# send2reply.train(corpus)
# send2reply.save('../tmp/send2reply.graph')
send2reply.load('../tmp/send2reply.graph')
len(send2reply)

42025

In [7]:
from scipy.io import mmwrite
from scipy.io import mmread

# send_x = vectorizer.transform(send2reply.sends)
# reply_x = vectorizer.transform(send2reply.replies)
# mmwrite('../tmp/send_x.mtx', send_x)
# mmwrite('../tmp/reply_x.mtx', reply_x)

send_x = mmread('../tmp/send_x.mtx').tocsr()

## Full search Index

In [8]:
from retrieval_bot.db import FullSearchIndexer

send_indexer = FullSearchIndexer(send_x)

In [9]:
dist, idx = send_indexer.kneighbors(query=send_x[0], n_neighbors=10, max_distance=0.5)
print(idx)
print(dist)

[    0 17039   963]
[0.         0.29289322 0.29289322]


## Default Message

In [10]:
from retrieval_bot.engine import DefaultMessage

default_message = DefaultMessage()
for _ in range(5):
    print(default_message.get_random_message())

음...
음...
무슨 말인지 잘 모르겠어요
다시 말해줄래요?
음...


## Engine

In [11]:
from retrieval_bot.engine import InstanceBasedRetrievalEngine

vectorizer.verbose = False
send2reply.verbose = False

engine = InstanceBasedRetrievalEngine(
    vectorizer = vectorizer,
    send2reply = send2reply,
    send_indexer = send_indexer)

In [12]:
engine.default_message.get_random_message()

'응??'

In [14]:
engine.process('지금 어디냥? ')

['넌?', '집이지', '나왔어', '집', '응']